In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from imblearn import over_sampling, under_sampling
import matplotlib.pyplot as plt
import seaborn as sns

### скачаем данные в переменные для работы

In [69]:
data = pd.read_csv("gb-tutors-expected-math-exam-results/train.csv")
df=pd.read_csv('gb-tutors-expected-math-exam-results/test.csv')

### посмотрим на распределение целевой переменной и данные

In [70]:
data

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,0,40.0,0.0,1400.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,61
1,1,48.0,4.0,2850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,76
2,2,39.0,0.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,53
3,3,46.0,5.0,1400.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,54
4,4,43.0,1.0,1500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,57
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,44.0,0.0,1700.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,83
9996,9996,51.0,0.0,1700.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,58
9997,9997,34.0,1.0,1250.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,62
9998,9998,33.0,3.0,1100.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56


In [71]:
df.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
0,10000,46.0,3.0,1050.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10001,43.0,3.0,1850.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,52.0,1.0,1550.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10003,57.0,6.0,2900.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
4,10004,44.0,4.0,3150.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0


In [72]:
feature_names=data.keys()
feature_names2=df.keys()
feature_names

Index(['Id', 'age', 'years_of_experience', 'lesson_price', 'qualification',
       'physics', 'chemistry', 'biology', 'english', 'geography', 'history',
       'mean_exam_points'],
      dtype='object')

In [73]:
X = pd.DataFrame(data, columns=feature_names)
X_df=pd.DataFrame(df, columns=feature_names2)

### уберем колонку с id из тренировочных данных

In [74]:
X=X.drop(['Id'], axis=1)

In [75]:
### запишем целевое значение в переменную 'y'

In [76]:
y = X['mean_exam_points']
y

0       61
1       76
2       53
3       54
4       57
        ..
9995    83
9996    58
9997    62
9998    56
9999    52
Name: mean_exam_points, Length: 10000, dtype: int64

In [77]:
X=X.drop('mean_exam_points', axis=1)

### функции для вычисления ошибки

In [78]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

<!-- Нормализ  -->

In [79]:
# def min_max_scale(X):
#     return (X - X.min()) / (X.max() - X.min())

In [80]:
# X_norm = X.copy()
# X_norm = X_norm.astype(np.float64)
# X_st=min_max_scale(X_norm)

In [81]:
def standard_scale(X):
    mean = X.mean()
    std = X.std()
    return (X - mean) / std

In [82]:
X_st = X.copy()
X_st = X_st.astype(np.float64)
X_st = standard_scale(X)
X_st.describe()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,2.711165e-17,-2.819966e-17,-5.198619e-17,-5.876410e-16,-3.176792e-16,-3.645972e-16,1.374123e-16,-1.462014e-15,-1.804690e-15,-2.018996e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.844133e+00,-1.121084e+00,-2.856055e+00,-9.081567e-01,-7.745579e-01,-3.914769e-01,-3.508254e-01,-2.382051e-01,-1.821024e-01,-1.406479e-01
25%,-7.307375e-01,-1.121084e+00,-7.603642e-01,-9.081567e-01,-7.745579e-01,-3.914769e-01,-3.508254e-01,-2.382051e-01,-1.821024e-01,-1.406479e-01
50%,1.516672e-02,7.448314e-03,-3.793295e-01,3.540486e-01,-7.745579e-01,-3.914769e-01,-3.508254e-01,-2.382051e-01,-1.821024e-01,-1.406479e-01
75%,6.367535e-01,5.717145e-01,8.590331e-01,3.540486e-01,1.290930e+00,-3.914769e-01,-3.508254e-01,-2.382051e-01,-1.821024e-01,-1.406479e-01
max,2.750149e+00,4.521578e+00,4.288345e+00,2.878459e+00,1.290930e+00,2.554173e+00,2.850136e+00,4.197644e+00,5.490868e+00,7.109243e+00


In [83]:
n_features=X_st.shape[1]
n_features

10

### создадим матрицу из единиц и прибавим к тренировочному датасету для вычисления линейной регрессии с коэффициентом b0

In [84]:
n = X_st.shape[0]
ones=np.ones(n)
ones=pd.Series(ones)
ones.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [85]:
### переставим в начало столбец с единичками

In [86]:
# X_st['ones']=ones
X_st.insert(0, "one", ones)
X_st.head()

,one,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
0,1.0,-0.730737,-1.121084,-0.569847,-0.908157,1.290930,-0.391477,-0.350825,-0.238205,5.490868,-0.140648
1,1.0,0.263801,1.135981,2.192654,1.616254,1.290930,-0.391477,-0.350825,-0.238205,-0.182102,-0.140648
2,1.0,-0.855055,-1.121084,-0.950881,-0.908157,-0.774558,-0.391477,-0.350825,-0.238205,-0.182102,-0.140648
3,1.0,0.015167,1.700247,-0.569847,-0.908157,-0.774558,-0.391477,-0.350825,-0.238205,-0.182102,-0.140648
4,1.0,-0.357785,-0.556818,-0.379330,-0.908157,-0.774558,-0.391477,-0.350825,-0.238205,-0.182102,-0.140648


In [87]:
# y_pred_analytical = X_st @ W
# y_pred_analytical

In [ ]:
# def gradient_descent_reg_l2(X, y, iterations, eta=8e-4, reg=1e-3):
iterations=5000
eta=7e-3
reg=2e-2
W = np.random.randn(X_st.shape[1])
n = X_st.shape[0]

for i in range(0, iterations):
    y_pred = np.dot(X_st, W)
    err = calc_mse(y, y_pred)

    dQ = 2/n * X_st.T @ (y_pred - y) # градиент функции ошибки
    dReg = reg * W # градиент регуляризации

    W -= eta * (dQ + dReg)

    if i % (iterations / 100) == 0:
        print(f'Iter: {i}, weights: /n{W}, error {err}')

print(f'Final MSE: {calc_mse(y, np.dot(X_st, W))}')

Iter: 0, weights: /none                    1.807307
age                    0.062161
years_of_experience   -0.174142
lesson_price          -0.126583
qualification          0.743526
physics               -0.269409
chemistry              0.262825
biology               -0.723988
english               -1.274032
geography              0.500507
history                0.150980
dtype: float64, error 4160.090628144085
Iter: 50, weights: /none                    33.030576
age                    -0.038510
years_of_experience     0.676426
lesson_price            3.507504
qualification           4.594205
physics                 1.117085
chemistry               0.206773
biology                -0.135199
english                -0.537121
geography               0.323352
history                 0.101540
dtype: float64, error 1077.829458728617
Iter: 100, weights: /none                    48.349937
age                    -0.086064
years_of_experience     0.658610
lesson_price            4.442699
qualificat

Iter: 1000, weights: /none                    63.106198
age                    -0.099498
years_of_experience     0.312408
lesson_price            4.445404
qualification           6.609800
physics                 2.441769
chemistry               0.096158
biology                 0.453494
english                 0.206026
geography               0.132350
history                 0.060987
dtype: float64, error 97.45886345972524
Iter: 1050, weights: /none                    63.106218
age                    -0.099544
years_of_experience     0.312712
lesson_price            4.441289
qualification           6.613831
physics                 2.441694
chemistry               0.096199
biology                 0.453508
english                 0.206009
geography               0.132392
history                 0.061002
dtype: float64, error 97.45857527133873
Iter: 1100, weights: /none                    63.106228
age                    -0.099584
years_of_experience     0.312974
lesson_price            4.

In [ ]:
y_pred

In [ ]:
my_score = pd.DataFrame({
"Id": X_df['Id'],
"mean_exam_points": y_pred,
})
my_score



In [ ]:
my_score.to_csv('predictions3.csv', index=False, encoding='utf-8')

In [ ]:
from sklearn.metrics import r2_score
r2_score(y,y_pred)

In [ ]:
np.mean((y - y_pred) ** 2)